# Data collected from OP's website op-koti.fi. Initially I used webscraping technique to collect data. But using the API was more dynamic and easy to handle

In [1]:
import numpy as np
import pandas as pd
import requests
# import math
import re
from pandas import json_normalize

pd.options.display.max_columns = None
# display(df)

In [2]:
URL = 'https://op-koti.fi/api/apartments?mode=sale&featureGroup=apartment&orderBy=created&order=desc&offset=0&limit=4500'
# request the URL and parse the JSON
response = requests.get(URL)
response.raise_for_status() # raise exception if invalid response
data = response.json()['listings']

# normalizing the json data into a dataframe
df = json_normalize(data, max_level = 1)

In [3]:
df.head(10)

,id,kipinaId,listingType,commission,listingGroup,source,created,featureGroups,floor,year,imageUrl,showings,rooms,numberOfRooms,price,debtFreePrice,rentPerMonth,location.streetAddress,location.postalCode,location.postOffice,location.city,location.cityId,location.region,location.regionId,location.district,location.districtId,location.streetId,location.coordinates,housing.livingArea,housing.totalArea,housing.siteArea,housing.rooms,housing.numberOfRooms,pricing.price,pricing.debtFreePrice,livingArea.size,livingArea.displayUnit,totalArea.size,totalArea.displayUnit,siteArea.size,siteArea.displayUnit,virtualPresentationUrl,videoUrl,siteArea,livingArea,totalArea,newHousing.newHousingId,newHousing.name
0,689338,414041,91,O,82,Kipinä,2021-09-17T11:08:13.46Z,[apartment],0.0,1980.0,https://op-koti.fi/kuvat/689338/1ed6d9d28c2948...,[],3h+k+s,3.0,59000.00,59000.0,None,Käsityötie 6 C,63300,ALAVUS,Alavus,10,Alavus,70,Kaarti,13267.0,0a98359808c7ced24a60ab428ac6bbd5b9ff4489,"{'latitude': 62.58513, 'longitude': 23.59916}","{'size': 76, 'displayUnit': 'm2'}","{'size': 76, 'displayUnit': 'm2'}","{'size': 3779, 'displayUnit': 'm2'}",3h+k+s,3.0,59000.00,59000.0,76.0,m2,76.0,m2,3779.0,m2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,696999,421695,89,O,82,Kipinä,2021-09-17T11:08:13.107Z,[apartment],3.0,1978.0,https://op-koti.fi/kuvat/696999/a60bf60819369f...,[],"2h,k",2.0,104888.37,130000.0,None,Tuusulantie 37 C,04200,KERAVA,Kerava,245,Kerava,171,Untola,2174.0,bad61c2be56a0e75c0bfc8dc2a33538a14d9d841,"{'latitude': 60.4065, 'longitude': 25.09292}","{'size': 56, 'displayUnit': 'm2'}","{'size': 56, 'displayUnit': 'm2'}","{'size': 2896, 'displayUnit': 'm2'}","2h,k",2.0,104888.37,130000.0,56.0,m2,56.0,m2,2896.0,m2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,515532,440222,91,O,82,Kipinä,2021-09-17T11:08:12.336Z,[apartment],0.0,1982.0,https://op-koti.fi/kuvat/515532/e0c5250e2a01b5...,[],"5h, k, ph, s, khh, wc",5.0,91328.97,110000.0,None,Rivitie 4 A,92140,PATTIJOKI,Raahe,678,Raahe,345,Pattijoki,6104.0,a208c73dee967a52f249bf082be987ac939266bf,"{'latitude': 64.69064, 'longitude': 24.58378}","{'size': 116.5, 'displayUnit': 'm2'}","{'size': 116.5, 'displayUnit': 'm2'}","{'size': 6350, 'displayUnit': 'm2'}","5h, k, ph, s, khh, wc",5.0,91328.97,110000.0,116.5,m2,116.5,m2,6350.0,m2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,515499,440189,90,K,82,Kipinä,2021-09-17T11:08:10.74Z,[apartment],0.0,1960.0,https://op-koti.fi/kuvat/515499/3fb394ccec4265...,[],"4h,k,kh,s,ph,wc,vh,ullakot,kellaritilat",4.0,118000.00,NaN,None,Korpiahontie 28,41800,KORPILAHTI,Jyväskylä,179,Jyväskylä,138,"Korpilahti, Korpiaho",7225.0,dd516facdfa6fbcbc1a38cb12375a5eedc808071,"{'latitude': 62.02292, 'longitude': 25.54483}","{'size': 120, 'displayUnit': 'm2'}","{'size': 160, 'displayUnit': 'm2'}","{'size': 1850, 'displayUnit': 'm2'}","4h,k,kh,s,ph,wc,vh,ullakot,kellaritilat",4.0,118000.00,NaN,120.0,m2,160.0,m2,1850.0,m2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,515576,440266,89,O,82,Kipinä,2021-09-17T11:08:03.919Z,[apartment],2.0,1963.0,https://op-koti.fi/kuvat/515576/ada1a7ce733de3...,[],1h+kk,1.0,76451.00,89000.0,None,Viitaniementie 21 B,40720,JYVÄSKYLÄ,Jyväskylä,179,Jyväskylä,138,Viitaniemi,2308.0,45011a65f398b37e6d1688ab545faa986cee9dd6,"{'latitude': 62.25287, 'longitude': 25.73191}","{'size': 27, 'displayUnit': 'm2'}","{'size': 27, 'displayUnit': 'm2'}","{'size': 3641.5, 'displayUnit': 'm2'}",1h+kk,1.0,76451.00,89000.0,27.0,m2,27.0,m2,3641.5,m2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,515443,440133,90,K,83,Kipinä,2021-09-17T11:07:52.639Z,"[newHousing, apartment]",0.0,1926.0,https://op-koti.fi/kuvat/515443/414010701b48b7...,[],"alak:3h,k,wc, yläk: 2h,k,wc,aula",5.0,54000.00,NaN,None,Peipontie 12,48220,KOTKA,Kotka,285,Kotka,198,Metsola,2470.0,0d215788491758fe80afb7f8f43d788d1b0215ef,"{'latitude': 60.48123, 'longitude': 26.90548}","{'size': 145, 'displayUnit': 'm2'}","{'size': 215, 'displayUnit': 'm2'}","{'size': 946, 'displayUnit': 'm2'}","alak:3h,k,wc, yläk: 2h,k,wc,aula",5.0,54000.00,NaN,145.0,m2,215.0,m2,946.0,m2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,515323,440013,8

In [4]:
# df[df.id == '696611']
# display(df)

It can be seen above that the dataframe contains multiple columns that are irrelevant to us. Relevant columns are selected as shown below.

In [5]:
df.columns

Index(['id', 'kipinaId', 'listingType', 'commission', 'listingGroup', 'source',
       'created', 'featureGroups', 'floor', 'year', 'imageUrl', 'showings',
       'rooms', 'numberOfRooms', 'price', 'debtFreePrice', 'rentPerMonth',
       'location.streetAddress', 'location.postalCode', 'location.postOffice',
       'location.city', 'location.cityId', 'location.region',
       'location.regionId', 'location.district', 'location.districtId',
       'location.streetId', 'location.coordinates', 'housing.livingArea',
       'housing.totalArea', 'housing.siteArea', 'housing.rooms',
       'housing.numberOfRooms', 'pricing.price', 'pricing.debtFreePrice',
       'livingArea.size', 'livingArea.displayUnit', 'totalArea.size',
       'totalArea.displayUnit', 'siteArea.size', 'siteArea.displayUnit',
       'virtualPresentationUrl', 'videoUrl', 'siteArea', 'livingArea',
       'totalArea', 'newHousing.newHousingId', 'newHousing.name'],
      dtype='object')

In [6]:
cols = ['id', 'listingType', 'floor', 'year', 'rooms', 'numberOfRooms', 'price', 'debtFreePrice',
       'location.city', 'location.region', 'location.district','location.postalCode',
       'livingArea.size', 'totalArea.size']

df = df[cols]

Renaming the cols

In [7]:
df = df.rename(columns = {'location.city':'city','location.region':'region','location.district':'district','location.postalCode':'postalCode','livingArea.size':'livingArea','totalArea.size':'totalArea', 'year':'yearBuilt'})

Mapping the listingType column to the apporpriate type of house (extracted from the website manually)

In [8]:
lType = {89:'Kerrostalo',90:'Omakotitalo',91:'Rivitalo',92:'Paritalo',93:'Erillistalo',112:'Puutalo',113:'Luhtitalo',470:'Kytketty paritalo'}
df['listingType'] = df['listingType'].map(lType)

In [9]:
# some values in df.district are ''. So i decided to fill them with np.nan and then remove them altogether in the next step
df[df.district == ''] = np.nan

In [10]:
def count_NAs(df):
    for col in df:
        print(col," : ",df[col].isnull().sum())
        
# def homeSizeLabels(area):
#     """these ranges were calulated by using pd.cut(). 1=Smallest...4=Largest"""
#     if area in range(0,100):
#         return 1
#     elif area in range(100,175):
#         return 2
#     elif area in range(175,250):
#         return 3
#     else:
#         return 4 

In [11]:
# df = df.dropna(inplace = True)
count_NAs(df)

id  :  210
listingType  :  210
floor  :  608
yearBuilt  :  218
rooms  :  317
numberOfRooms  :  317
price  :  317
debtFreePrice  :  1435
city  :  210
region  :  210
district  :  210
postalCode  :  210
livingArea  :  317
totalArea  :  317


There are some columns with null values for debtFreePrice. Those values are replaced by price. 

In [12]:
df.loc[df.debtFreePrice.isnull() & df.price.notnull(),['debtFreePrice']] = df.price

Converting the columns to string and integer(from float) in order to remove the .0 at the end of figures

In [13]:
# df['id'] = df['id'].astype(str)
# df['floor'] = df['floor'].astype(int)
# df['numberOfRooms'] = df['numberOfRooms'].astype(int)
# df['yearBuilt'] = df['yearBuilt'].astype(int)
# df['postalCode'] = df['postalCode'].astype(str)

Calculating building age from 'yearBuilt'. Some buildings are supposed to be completed in the future. So any building that gets completed in the future year are also grouped into '1 year old buildings'

In [14]:
df['buildingAge'] = [1 if x > 2021 else (pd.to_datetime('today').year - pd.to_datetime(x, format = "%Y").year) for x in df.yearBuilt]
# df['centrum'] = [1 if x == 'Keskusta' else 0 for x in df.district]
# df['howBig'] = [homeSizeLabels(x) for x in df.livingArea]

In [15]:
# hideRows = ['listingGroup', 'yearBuilt', 'imageUrl', 'streetAddress', 'postalCode', 'region', 'link']
df.drop('yearBuilt', axis = 1, inplace = True)
df.reset_index(drop = True)

,id,listingType,floor,rooms,numberOfRooms,price,debtFreePrice,city,region,district,postalCode,livingArea,totalArea,buildingAge
0,689338,Rivitalo,0.0,3h+k+s,3.0,59000.00,59000.0,Alavus,Alavus,Kaarti,63300,76.0,76.0,41.0
1,696999,Kerrostalo,3.0,"2h,k",2.0,104888.37,130000.0,Kerava,Kerava,Untola,04200,56.0,56.0,43.0
2,515532,Rivitalo,0.0,"5h, k, ph, s, khh, wc",5.0,91328.97,110000.0,Raahe,Raahe,Pattijoki,92140,116.5,116.5,39.0
3,515499,Omakotitalo,0.0,"4h,k,kh,s,ph,wc,vh,ullakot,kellaritilat",4.0,118000.00,118000.0,Jyväskylä,Jyväskylä,"Korpilahti, Korpiaho",41800,120.0,160.0,61.0
4,515576,Kerrostalo,2.0,1h+kk,1.0,76451.00,89000.0,Jyväskylä,Jyväskylä,Viitaniemi,40720,27.0,27.0,58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3614,kiinteisto-oy-joensuun-koskikatu-9,Kerrostalo,NaN,None,NaN,NaN,NaN,Joensuu,Joensuu,Keskusta,80100,NaN,NaN,6.0
3615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3616,604375,Kerrostalo,1.0,"1h,k",1.0,18500.00,18500.0,Virrat,Virrat,Nallela,34800,30.0,30.0,51.0
3617,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The values in the 'rooms' column are separated using , or + or . or * and need to be cleaned

In [16]:
df.dropna(inplace = True)
count_NAs(df)

id  :  0
listingType  :  0
floor  :  0
rooms  :  0
numberOfRooms  :  0
price  :  0
debtFreePrice  :  0
city  :  0
region  :  0
district  :  0
postalCode  :  0
livingArea  :  0
totalArea  :  0
buildingAge  :  0


In [17]:
def clean_rooms(item):
    delimiters = "+", ","
    # >>> example = "stackoverflow (c) is awesome... isn't it?"
    regexPattern = '|'.join(map(re.escape, delimiters))
    return([re.sub('\\+|,|\\.|\\*| ', '', x) for x in re.split(regexPattern, item)])

df.rooms = [clean_rooms(x) for x in df.rooms]

These functions take a single entry from df.rooms and detects the features. These features are then represented with new columns using 1 or 0 denoting the presence or absence of the feature

In [18]:
def detect_centrum(postalCode):
    if postalCode[2:] == '100':
        return 1
    else:
        return 0

df['centrum'] = [detect_centrum(x) for x in df.postalCode]

def detect_sauna(room): #v4
    if 's' in room or str(room).find('sauna') != -1:
        return 1
    else:
        return 0
df['hasSauna'] = [detect_sauna(x) for x in df.rooms]

def detect_balcony(room): #v3
    if 'p' in room or str(room).find('parv') != -1:
        return 1
    else:
        return 0
df['hasBalcony'] = [detect_balcony(x) for x in df.rooms]

def detect_parking(room): #v3
    if 'ak' in room or 'at' in room or str(room).find('auto') != -1:
        return 1
    else:
        return 0
df['hasParking'] = [detect_parking(x) for x in df.rooms]

def detect_walk_in_closet(room): #v3
    if 'v' in room or 'vh' in room or str(room).find('vaate') != -1:
        return 1
    else:
        return 0
df['hasWalkInCloset'] = [detect_walk_in_closet(x) for x in df.rooms]

def detect_storage_room(room):
    if str(room).find('var') != -1:
        return 1
    else:
        return 0
df['hasStorageRoom'] = [detect_storage_room(x) for x in df.rooms]

In [19]:
df.drop('rooms', axis = 1, inplace = True)

In [20]:
df['id'] = df['id'].astype(str)
df['floor'] = df['floor'].astype(int)
df['numberOfRooms'] = df['numberOfRooms'].astype(int)
df['postalCode'] = df['postalCode'].astype(str)

In [21]:
df.head()

,id,listingType,floor,numberOfRooms,price,debtFreePrice,city,region,district,postalCode,livingArea,totalArea,buildingAge,centrum,hasSauna,hasBalcony,hasParking,hasWalkInCloset,hasStorageRoom
0,689338,Rivitalo,0,3,59000.00,59000.0,Alavus,Alavus,Kaarti,63300,76.0,76.0,41.0,0,1,0,0,0,0
1,696999,Kerrostalo,3,2,104888.37,130000.0,Kerava,Kerava,Untola,04200,56.0,56.0,43.0,0,0,0,0,0,0
2,515532,Rivitalo,0,5,91328.97,110000.0,Raahe,Raahe,Pattijoki,92140,116.5,116.5,39.0,0,1,0,0,0,0
3,515499,Omakotitalo,0,4,118000.00,118000.0,Jyväskylä,Jyväskylä,"Korpilahti, Korpiaho",41800,120.0,160.0,61.0,0,1,0,0,1,0
4,515576,Kerrostalo,2,1,76451.00,89000.0,Jyväskylä,Jyväskylä,Viitaniemi,40720,27.0,27.0,58.0,0,0,0,0,0,0


In [22]:
set(df.listingType)

{'Erillistalo',
 'Kerrostalo',
 'Kytketty paritalo',
 'Luhtitalo',
 'Omakotitalo',
 'Paritalo',
 'Puutalo',
 'Rivitalo'}

Exporting the dataframe to a csv file

In [23]:
df.to_csv(f'op-koti.csv', sep=',', encoding='utf-8', index=False)